In [4]:
import os
import sys
import csv
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix as cmat
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as report  
from sklearn.model_selection import *
from sklearn.feature_selection import *
import matplotlib.pyplot as plt
import time
import random
import numpy
from datetime import datetime
import pandas as pd
import requests
import re
import xarray as xr
import pickle
import glob

In [6]:
# Read data from HOT
directory = '../data/hot_rosette/small'

file_list = glob.glob('../data/hot_rosette/*.csv')
fields = ['STNNBR', 'DATE', 'TIME', 'LATITUDE', 'LONGITUDE', 'CTDPRS', 'CTDTMP', 'CTDSAL', 'CTDOXY', 'CTDSAL_FLAG_W',
         'OXYGEN', 'OXYGEN_FLAG_W', 'SILCAT', 'SILCAT_FLAG_W', 'PHSPHT', 'PHSPHT_FLAG_W', 'NO2+NO3', 'NO2+NO3_FLAG_W']

df = pd.DataFrame(columns=fields)
df0 = pd.DataFrame(columns=fields)

for x in file_list:
    fil_name = x
    a = pd.read_csv(fil_name, skiprows=[0], comment='#')
    a = a[fields]
    a = a[a.STNNBR==2]
    a = a[a.CTDSAL_FLAG_W==2]
    df0=df0.append(a, ignore_index=True)
    a = a[a['NO2+NO3_FLAG_W']==2]
    #print(a.shape)
    df=df.append(a, ignore_index=True)
print('Before data cleaning:')
print(df0.shape)
print('After data cleaning:')
print(df.shape)

KeyError: "['STNNBR' 'DATE' 'TIME' 'LATITUDE' 'LONGITUDE' 'CTDPRS' 'CTDTMP' 'CTDSAL'\n 'CTDOXY' 'CTDSAL_FLAG_W' 'OXYGEN' 'OXYGEN_FLAG_W' 'SILCAT' 'SILCAT_FLAG_W'\n 'PHSPHT' 'PHSPHT_FLAG_W' 'NO2+NO3' 'NO2+NO3_FLAG_W'] not in index"

In [40]:
directory = '../data/hot_rosette/small'

header_line = 'EXPOCODE,SECT_ID,STNNBR,CASTNO,SAMPNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,LONGITUDE,DEPTH,CTDPRS,CTDTMP,CTDSAL,CTDSAL_FLAG_W,SALNTY,SALNTY_FLAG_W,CTDOXY,CTDOXY_FLAG_W,OXYGEN,OXYGEN_FLAG_W,SILCAT,SILCAT_FLAG_W,PHSPHT,PHSPHT_FLAG_W,REVPRS,CTDRAW,THETA,NO2+NO3,NO2+NO3_FLAG_W,REVTMP\n'

header_length = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        lines = open(directory+'/'+filename).readlines()
        #print(lines)
        header_length.append(lines.index(header_line)+1)
header_length

ValueError: 'EXPOCODE,SECT_ID,STNNBR,CASTNO,SAMPNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,LONGITUDE,DEPTH,CTDPRS,CTDTMP,CTDSAL,CTDSAL_FLAG_W,SALNTY,SALNTY_FLAG_W,CTDOXY,CTDOXY_FLAG_W,OXYGEN,OXYGEN_FLAG_W,SILCAT,SILCAT_FLAG_W,PHSPHT,PHSPHT_FLAG_W,REVPRS,CTDRAW,THETA,NO2+NO3,NO2+NO3_FLAG_W,REVTMP\n' is not in list

In [51]:
directory = '../data/hot_rosette/short_headers'
filename = directory+'/'+'19_33KB20111218_hy1.csv'
#fieldnames = ['Pressure','Temperature','Salinity','Sigma-t','Oxygen_v','Oxygen_m','Fluorescence']

ross_data = pd.DataFrame()

ross_data = pd.read_csv(filename,header=1)
ross_data

,EXPOCODE,SECT_ID,STNNBR,CASTNO,SAMPNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,...,SILCAT,SILCAT_FLAG_W,PHSPHT,PHSPHT_FLAG_W,REVPRS,CTDRAW,THETA,NO2+NO3,NO2+NO3_FLAG_W,REVTMP
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,UMOL/KG,NaN,UMOL/KG,NaN,DBAR,DBAR,ITS-90,UMOL/KG,NaN,DEG C
1,33KB20111218,PRS2,2.0,1.0,24.0,8024.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,24,24.1939,-999.00,9.0,-999.0
2,33KB20111218,PRS2,2.0,1.0,23.0,8023.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,24,24.1936,-999.00,9.0,-999.0
3,33KB20111218,PRS2,2.0,1.0,22.0,8022.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,25,24.1937,-999.00,9.0,-999.0
4,33KB20111218,PRS2,2.0,1.0,21.0,8021.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,23,24.1950,-999.00,9.0,-999.0
5,33KB20111218,PRS2,2.0,1.0,20.0,8020.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,25,24.1966,-999.00,9.0,-999.0
6,33KB20111218,PRS2,2.0,1.0,19.0,8019.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,43,24.1986,-999.00,9.0,-999.0
7,33KB20111218,PRS2,2.0,1.0,18.0,8018.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,43,24.1990,-999.00,9.0,-999.0
8,33KB20111218,PRS2,2.0,1.0,17.0,8017.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,44,24.1995,-999.00,9.0,-999.0
9,33KB20111218,PRS2,2.0,1.0,16.0,8016.0,2.0,20111219.0,1045.0,22.749833,...,-999.00,9.0,-999.00,9.0,-999.0,44,24.1991,-999.00,9.0,-999.0


In [52]:
directory = '../data/hot_rosette/short_headers'


for filename in os.listdir(directory):
    each_data = pd.read_csv(directory+'/'+filename, error_bad_lines=False,header=1)
    ross_data = pd.concat([ross_data,each_data],axis=0)

ross_data

Skipping line 6: expected 1 fields, saw 32
Skipping line 7: expected 1 fields, saw 32
Skipping line 8: expected 1 fields, saw 32
Skipping line 9: expected 1 fields, saw 32
Skipping line 10: expected 1 fields, saw 32
Skipping line 11: expected 1 fields, saw 32
Skipping line 12: expected 1 fields, saw 32
Skipping line 13: expected 1 fields, saw 32
Skipping line 14: expected 1 fields, saw 32
Skipping line 15: expected 1 fields, saw 32
Skipping line 16: expected 1 fields, saw 32
Skipping line 17: expected 1 fields, saw 32
Skipping line 18: expected 1 fields, saw 32
Skipping line 19: expected 1 fields, saw 32
Skipping line 20: expected 1 fields, saw 32
Skipping line 21: expected 1 fields, saw 32
Skipping line 22: expected 1 fields, saw 32
Skipping line 23: expected 1 fields, saw 32
Skipping line 24: expected 1 fields, saw 32
Skipping line 25: expected 1 fields, saw 32
Skipping line 26: expected 1 fields, saw 32
Skipping line 27: expected 1 fields, saw 32
Skipping line 28: expected 1 fields,

,#code : jjward hyd_to_exchange.pl,BTLNBR,BTLNBR_FLAG_W,CASTNO,CTDOXY,CTDOXY_FLAG_W,CTDPRS,CTDRAW,CTDSAL,CTDSAL_FLAG_W,...,REVTMP,SALNTY,SALNTY_FLAG_W,SAMPNO,SECT_ID,SILCAT,SILCAT_FLAG_W,STNNBR,THETA,TIME
0,NaN,NaN,NaN,NaN,UMOL/KG,NaN,DBAR,DBAR,PSS-78,NaN,...,DEG C,PSS-78,NaN,NaN,NaN,UMOL/KG,NaN,NaN,ITS-90,NaN
1,NaN,8024.0,2.0,1.0,207.5,1.0,25.3,24,35.2841,2.0,...,-999.0,-999.0000,9.0,24.0,PRS2,-999.00,9.0,2.0,24.1939,1045.0
2,NaN,8023.0,2.0,1.0,207.6,1.0,25.3,24,35.2841,2.0,...,-999.0,-999.0000,9.0,23.0,PRS2,-999.00,9.0,2.0,24.1936,1045.0
3,NaN,8022.0,2.0,1.0,207.6,1.0,25.3,25,35.2843,2.0,...,-999.0,-999.0000,9.0,22.0,PRS2,-999.00,9.0,2.0,24.1937,1045.0
4,NaN,8021.0,2.0,1.0,207.5,1.0,24.9,23,35.2842,2.0,...,-999.0,-999.0000,9.0,21.0,PRS2,-999.00,9.0,2.0,24.1950,1045.0
5,NaN,8020.0,2.0,1.0,207.4,1.0,25.4,25,35.2839,2.0,...,-999.0,-999.0000,9.0,20.0,PRS2,-999.00,9.0,2.0,24.1966,1045.0
6,NaN,8019.0,2.0,1.0,207.3,1.0,43.7,43,35.2835,2.0,...,-999.0,-999.0000,9.0,19.0,PRS2,-999.00,9.0,2.0,24.1986,1045.0
7,NaN,8018.0,2.0,1.0,207.2,1.0,43.7,43,35.2836,2.0,...,-999.0,-999.0000,9.0,18.0,PRS2,-999.00,9.0,2.0,24.1990,1045.0
8,NaN,8017.0,2.0,1.0,207.0,1.0,43.7,44,35.2835,2.0,...,-999.0,-999.0000,9.0,17.0,PRS2,-999.00,9.0,2.0,24.1995,1045.0
9,NaN,8016.0,2.0,1.0,206.9,1.0,43.8,44,35.2834,2.0,...,-999.0,-999.0000,9.0,16.0,PRS2,-999.00,9.0,2.0,24.1991,1045.0


In [58]:
ross_data = ross_data.fillna(-999.0)
ross_data.astype(bool).sum(axis=0)

#code : jjward hyd_to_exchange.pl     8419
BTLNBR                                8419
BTLNBR_FLAG_W                         8419
CASTNO                                8419
CTDOXY                                8419
CTDOXY_FLAG_W                         8419
CTDPRS                                8419
CTDRAW                                8419
CTDSAL                                8419
CTDSAL_FLAG_W                         8419
CTDTMP                                8419
DATE                                  8419
DEPTH                                 8419
EXPOCODE                              8419
LATITUDE                              8419
LONGITUDE                             8419
NO2+NO3                               8419
NO2+NO3_FLAG_W                        8419
OXYGEN                                8419
OXYGEN_FLAG_W                         8419
PHSPHT                                8419
PHSPHT_FLAG_W                         8419
REVPRS                                8419
REVTMP     

In [60]:
ross_data.to_csv(directory+'/'+'ross_data_consolidated.csv',index=False)

In [66]:
ross_data = ross_data.replace(-999,np.nan)
ross_data = ross_data.replace('-999.0',np.nan,regex=True)
ross_data = ross_data.replace('-999.00',np.nan)
ross_data = ross_data.replace('-999.0000',np.nan)

In [67]:
ross_data

,#code : jjward hyd_to_exchange.pl,BTLNBR,BTLNBR_FLAG_W,CASTNO,CTDOXY,CTDOXY_FLAG_W,CTDPRS,CTDRAW,CTDSAL,CTDSAL_FLAG_W,...,REVTMP,SALNTY,SALNTY_FLAG_W,SAMPNO,SECT_ID,SILCAT,SILCAT_FLAG_W,STNNBR,THETA,TIME
0,NaN,NaN,NaN,NaN,UMOL/KG,NaN,DBAR,DBAR,PSS-78,NaN,...,DEG C,PSS-78,NaN,NaN,NaN,UMOL/KG,NaN,NaN,ITS-90,NaN
1,NaN,8024.0,2.0,1.0,207.5,1.0,25.3,24,35.2841,2.0,...,NaN,NaN,9.0,24.0,PRS2,NaN,9.0,2.0,24.1939,1045.0
2,NaN,8023.0,2.0,1.0,207.6,1.0,25.3,24,35.2841,2.0,...,NaN,NaN,9.0,23.0,PRS2,NaN,9.0,2.0,24.1936,1045.0
3,NaN,8022.0,2.0,1.0,207.6,1.0,25.3,25,35.2843,2.0,...,NaN,NaN,9.0,22.0,PRS2,NaN,9.0,2.0,24.1937,1045.0
4,NaN,8021.0,2.0,1.0,207.5,1.0,24.9,23,35.2842,2.0,...,NaN,NaN,9.0,21.0,PRS2,NaN,9.0,2.0,24.1950,1045.0
5,NaN,8020.0,2.0,1.0,207.4,1.0,25.4,25,35.2839,2.0,...,NaN,NaN,9.0,20.0,PRS2,NaN,9.0,2.0,24.1966,1045.0
6,NaN,8019.0,2.0,1.0,207.3,1.0,43.7,43,35.2835,2.0,...,NaN,NaN,9.0,19.0,PRS2,NaN,9.0,2.0,24.1986,1045.0
7,NaN,8018.0,2.0,1.0,207.2,1.0,43.7,43,35.2836,2.0,...,NaN,NaN,9.0,18.0,PRS2,NaN,9.0,2.0,24.1990,1045.0
8,NaN,8017.0,2.0,1.0,207.0,1.0,43.7,44,35.2835,2.0,...,NaN,NaN,9.0,17.0,PRS2,NaN,9.0,2.0,24.1995,1045.0
9,NaN,8016.0,2.0,1.0,206.9,1.0,43.8,44,35.2834,2.0,...,NaN,NaN,9.0,16.0,PRS2,NaN,9.0,2.0,24.1991,1045.0


In [68]:
 ross_data.count()

#code : jjward hyd_to_exchange.pl       16
BTLNBR                                8359
BTLNBR_FLAG_W                         8359
CASTNO                                8359
CTDOXY                                8381
CTDOXY_FLAG_W                         8359
CTDPRS                                8381
CTDRAW                                8381
CTDSAL                                8381
CTDSAL_FLAG_W                         8359
CTDTMP                                8381
DATE                                  8359
DEPTH                                 8381
EXPOCODE                              8381
LATITUDE                              8359
LONGITUDE                             8359
NO2+NO3                               1200
NO2+NO3_FLAG_W                        8359
OXYGEN                                2055
OXYGEN_FLAG_W                         8359
PHSPHT                                1200
PHSPHT_FLAG_W                         8359
REVPRS                                  22
REVTMP     